# Analisis geografico #

### Importo librerias 

In [320]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from geotext import GeoText

### Carga del DataFrame

In [321]:
tweets = pd.read_csv("Archivos/train.csv")

In [322]:
tweets

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


### Análisis de la información dada en la columna location

La ubicacion de los diferentes tweets se indica en la columna location. Veamos un poco mejor el contenido de 
dicha columna para tener una idea general de la información con la que contamos.

In [323]:
tweets['location'].value_counts()

USA                     104
New York                 71
United States            50
London                   45
Canada                   29
                       ... 
Washington, DC 20009      1
Pontevedra, Galicia       1
Beit El - Israel          1
Around the world          1
Anaheim                   1
Name: location, Length: 3341, dtype: int64

In [324]:
tweets['location'].isnull().value_counts()

False    5080
True     2533
Name: location, dtype: int64

In [325]:
len(tweets['location'])

7613

Se observa una gran cantidad de tweets para los cuales no concemos su ubicación. Por otra parte la columna location se encuentra desorganizada ya que para algunos tweets se conoce la ciudad de origen, para otros el país y en otros simbolos extraños que no tienen sentido. Intentaremos conocer, en base a la información que conocemos,para cada tweet su ubicación.

Utilizaremos una nueva columna real_location  que nos dirá para cada tweet la ubicación obtenida

Intentaremos matchear cada string de la columna location con la ubicación correspondiente para limpiar un poco esta información. Para ello, descartamos todos los registros que será imposible ubicar por no contener información en 
esta columna. Para aquellos tweets colocamos en su columna real_location "NO" para asi ya no trabajar con ellos.

In [326]:
nulos=tweets['location'].isnull()
tweets['real_location']=nulos.apply(lambda x: "NO" if x == True else "-")

In [327]:
tweets['real_location'].value_counts()

-     5080
NO    2533
Name: real_location, dtype: int64

Ahora nos quedan 5080 strings que pueden corresponderse o no con una ubicación real.

In [328]:
restantes=tweets[tweets['real_location']!="NO"]

*** Primer reconocimiento ***

Iremos reconociendo poco a poco las ciudades, para hacer un primer reconocimeiento nos ayudaremos con la libreria Geotext. No tiene un gran funcionamiento ya que en algunos casos no reconoce correctamente los paises pero sirve como para determinar algunas ubicaciones.

Usaremos una función que dado un string nos permita localizar su ciudad o su país

In [329]:
def localizar(s):
    places = GeoText(str(s))
    if len(places.cities)!= 0:
        return places.cities[0]
    elif len(places.countries)!=0:
        return places.countries[0]
    else:
        return "-"
    

In [330]:
tweets.loc[tweets['real_location']!= "NO",'real_location']=restantes['location'].apply(localizar)

In [331]:
tweets['real_location'].value_counts()

NO               2533
-                2441
New York          122
London             99
United States      59
                 ... 
Eagle Pass          1
Newton              1
Sunnyvale           1
Montgomery          1
Ashford             1
Name: real_location, Length: 788, dtype: int64

In [350]:
restantes=tweets.loc[tweets['real_location']== "-"].sum()

Ahora nos queda ubicar 2441 tweets. Veamos cual es el estilo de sus locations.

In [335]:
restantes['location'].value_counts().sum

<bound method Series.sum of USA                            104
UK                              27
Worldwide                       19
California                      17
California, USA                 15
                              ... 
SEA Server                       1
Brum/Lestah                      1
Poconos                          1
Hospital, bc of SKH vid.         1
Click the link below, okay       1
Name: location, Length: 1854, dtype: int64>

Vamos a intentar ubicar paises a partir de su codigo ISO 3166-1. SE usaran las variantes
ISO 3166-1 alfa-3, sistema de códigos tres letras.
ISO 3166-1 alfa-2, sistema de códigos de dos letras.
Si los nombres de alguno de los resgistros restanes contiene alguno de estos códigos se le asginará el nombre del país. 

In [336]:
iso_paises=pd.read_csv('Archivos/iso_paises.csv')

In [337]:
iso_paises

,Country,iso2,iso3
0,Afghanistan,AF,AFG
1,Albanie,AL,ALB
2,Allemagne,DE,DEU
3,Andorra,AD,AND
4,Angola,AO,AGO
...,...,...,...
243,Wallis et Futuna,WF,WLF
244,Yémen,YE,YEM
245,Djibouti,DJ,DJI
246,Zambie,ZM,ZMB


In [345]:
def localizar_2(s):
    s=str(s)
    
    for index,row in iso_paises.iterrows():
        if str(row["iso2"]) in s or str(row["iso3"]) in s:
            return str(row["Country"])
        elif "UK" in  s:
            return "United Kingdom"
    return "-"
    

In [346]:
tweets.loc[tweets['real_location']=="-",'real_location']=restantes['location'].apply(localizar_2)

In [347]:
restantes=tweets.loc[tweets['real_location']== "-"]

In [348]:
restantes['location'].value_counts().sum()

1861

In [349]:
restantes['location'].head(60)

37                     World Wide!!
39                   Paranaque City
40                   Live On Webcam
42                        milky way
48                   Live On Webcam
49                         England.
50         Sheffield Township, Ohio
58                   hollywoodland 
62      Twitter Lockout in progress
72           Walker County, Alabama
74                   North Carolina
76                    Norf Carolina
78                   North Carolina
81            Your Sister's Bedroom
84             South Bloomfield, OH
93                           global
102        Instagram - @heyimginog 
103                             304
105                             304
107                             304
108        Instagram - @heyimginog 
109                             304
111        Somewhere Only We Know ?
117                       dope show
125                             304
126                             304
127                  marysville ca 
128                         

In [ ]:
### Ver como seguir :)